In [2]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn import set_config

import xgboost as xgb

from data_to_csv import df_to_csv
from utils import generate_description

df_1 = pd.read_csv(folder_path + '/SBA_Cleaned_1.csv')
df_1.dropna(inplace=True)
generate_description(df_1)
y = df_1['Approve']
X = df_1.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")

param_grid = {
    'model__n_estimators': [100, 200],  # Adjust according to your preference
    'model__max_depth': [5, 10],  # Adjust according to your preference
    'model__learning_rate': [0.1, 0.01],  # Adjust according to your preference
    'model__subsample': [0.8, 0.9],  # Adjust according to your preference
    'model__colsample_bytree': [0.8, 0.9]  # Adjust according to your preference
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)
f1_macro = f1_score(y_test, predictions, average='macro')
print("F1-score macro sur l'ensemble de test :", f1_macro)


Meilleurs hyperparamètres trouvés :
{'model__colsample_bytree': 0.9, 'model__learning_rate': 0.1, 'model__max_depth': 10, 'model__n_estimators': 200, 'model__subsample': 0.9}
Meilleure précision trouvée : 0.9078974577334051
F1-score macro sur l'ensemble de test : 0.9115407980778877
